# Usage Example

## Importing the NonPredictive class from the non-predictive module:

In [1]:
import sys
import pandas as pd
sys.path.append('..')
from glsm.non_predictive.model import NonPredictive
from glsm.non_predictive.features import Feature

## Instance the model and add the features:

In [2]:
model = NonPredictive()

feature_a = Feature(
    name="Monthly Users",
    weight=0.5,
    options_df=pd.DataFrame([
            {"label": "Up to 50K", "is_ICP": False, "points": 0},
            {"label": "50K - 100K", "is_ICP": True},
            {"label": "100K - 200K", "is_ICP": False},
            {"label": "More than 200K", "is_ICP": False}
        ])
)
feature_b = Feature(
    name="Industry",
    weight=0.25,
    options_df=pd.DataFrame([
            {"label": "Other", "is_ICP": False},
            {"label": "Agriculture", "is_ICP": False},
            {"label": "Transportation", "is_ICP": False},
            {"label": "Healthcare", "is_ICP": False},
            {"label": "Manufacturing", "is_ICP": False},
            {"label": "Education", "is_ICP": True},
            {"label": "Finance", "is_ICP": True},
            {"label": "Technology", "is_ICP": True},
            {"label": "Retail", "is_ICP": False},
            {"label": "Telecom", "is_ICP": False},
        ])
)

feature_c = Feature(
    name="Mkt Investment",
    weight=1,
    options_df=pd.DataFrame({
        'label': ['Up to $50K', '50k - $100K', '100k - $200K', '$200K - $300K', '$300K - $400K', 'More than $400K'],
        'is_ICP': [False, True, False, False, False, False],
        'points': [0, 0, 0, 0, 0, 0]
    })
)

model.add_features([feature_a, feature_b, feature_c])

## Points auto assignment
You may set preview=True to check the proposed changes

In [3]:
model.auto_assign_points()

,label,is_ICP,points,Feature Name
0,Up to 50K,False,0.0,Monthly Users
1,50K - 100K,True,50.0,Monthly Users
2,100K - 200K,False,75.0,Monthly Users
3,More than 200K,False,100.0,Monthly Users
4,Other,False,0.0,Industry
5,Agriculture,False,10.0,Industry
6,Transportation,False,20.0,Industry
7,Healthcare,False,30.0,Industry
8,Manufacturing,False,40.0,Industry
9,Education,True,50.0,Industry


## Compute lambda values for a single lead:

In [4]:
lead = { 
        "Monthly Users": "50K - 100K",
        "Industry": "Technology",
        "Mkt Investment": "$300K - $400K",
    }
    
lambda_value = model.compute_lambda(lead)
lambda_value

78.57

## Read leads from a csv file and compute the lead score:

In [5]:
import pandas as pd

df = pd.read_csv('leads.csv')
df['lambda'] = df.apply(
    lambda row: model.compute_lambda(row.to_dict()),
    axis=1
)
df

,Monthly Users,Industry,Mkt Investment,lambda
0,50K - 100K,Technology,$300K - $400K,78.57
1,50K - 100K,Retail,100k - $200K,60.71
2,Up to 50K,Agriculture,50k - $100K,38.57


## Assess qualification for each lead:

In [6]:
df['qualified'] = df.apply(
    lambda row: model.assess_qualification(row.to_dict()),
    axis=1
)
df

,Monthly Users,Industry,Mkt Investment,lambda,qualified
0,50K - 100K,Technology,$300K - $400K,78.57,True
1,50K - 100K,Retail,100k - $200K,60.71,True
2,Up to 50K,Agriculture,50k - $100K,38.57,False


## Removing features

In [7]:
model.remove_features(['Mkt Investment'])
len(model.features)

2

### Compute new lambda after removing a feature:

In [8]:
# df with both lambdas and delta_percent
df['new_lambda'] = df.apply(
    lambda row: model.compute_lambda(row.to_dict()),
    axis=1
)

df['delta_percent'] = (df['new_lambda'] / df['lambda'] - 1)*100

df.head()


,Monthly Users,Industry,Mkt Investment,lambda,qualified,new_lambda,delta_percent
0,50K - 100K,Technology,$300K - $400K,78.57,True,50.0,-36.362479
1,50K - 100K,Retail,100k - $200K,60.71,True,55.0,-9.405370
2,Up to 50K,Agriculture,50k - $100K,38.57,False,2.0,-94.814623
